# Retrieval-Augmented Generation (RAG) System using Gemini API
This notebook demonstrates a complete RAG pipeline using:

- Gemini API (Google Generative AI) for embeddings and text generation  
- FAISS for vector search  
- Local text corpus .txt files under ./data/

In [5]:
# Install dependencies (run this once per environment)
!pip install -q google-generativeai faiss-cpu numpy pandas

!pip install -U google-generativeai

!python -m pip install --upgrade pip

In [ ]:
import os
import faiss
import numpy as np
import pandas as pd
import google.generativeai as genai
from pathlib import Path
from typing import List, Dict

# Configure Gemini API key
# Get your key from https://aistudio.google.com/app/apikey
os.environ["GOOGLE_API_KEY"] = "Use you key"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])



In [15]:
def load_docs_from_dir(data_dir: str = "data") -> List[Dict]:
    data_path = Path(data_dir)
    texts = []
    for f in data_path.rglob("*.txt"):
        with open(f, "r", encoding="utf-8", errors="ignore") as file:
            content = file.read().strip()
            if content:
                texts.append({"source": str(f), "text": content})
    return texts

docs = load_docs_from_dir("data")
if not docs:
    raise ValueError("No .txt files found in ./data/. Please add text documents to proceed.")

In [16]:
def make_chunks(texts: List[Dict], chunk_size: int = 750, overlap: int = 100) -> List[Dict]:
    chunks = []
    for doc in texts:
        text = doc["text"]
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append({"source": doc["source"], "text": chunk})
            start += chunk_size - overlap
    return chunks

texts = make_chunks(docs)
print(f"Loaded {len(docs)} docs -> {len(texts)} chunks.")

Loaded 2 docs -> 2 chunks.


In [17]:
def embed_texts(texts: List[str], model: str = "models/text-embedding-004") -> np.ndarray:
    all_embs = []
    for t in texts:
        result = genai.embed_content(model=model, content=t)
        all_embs.append(result["embedding"])
    return np.array(all_embs, dtype=np.float32)

emb_matrix = embed_texts([t["text"] for t in texts])
print(f"Generated embeddings: {emb_matrix.shape}")

Generated embeddings: (2, 768)


In [18]:
index = faiss.IndexFlatIP(emb_matrix.shape[1])
faiss.normalize_L2(emb_matrix)
index.add(emb_matrix)
print(f"FAISS index built with {index.ntotal} vectors.")

FAISS index built with 2 vectors.


In [ ]:
def retrieve(query: str, k: int = 3) -> List[Dict]:
    q_emb = genai.embed_content(model="models/text-embedding-004", content=query)["embedding"]
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)
    faiss.normalize_L2(q_emb)
    scores, indices = index.search(q_emb, k)
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({"score": float(scores[0][i]), "text": texts[idx]["text"], "meta": texts[idx]})
    return results

def rag_answer(question: str, k: int = 3) -> Dict:
    retrieved = retrieve(question, k=k)
    context = "\n\n".join([r["text"] for r in retrieved])
    prompt = f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {question}\nAnswer:"
    model = genai.GenerativeModel("models/gemini-2.0-flash") 
    response = model.generate_content(prompt)
    return {"answer": response.text, "snippets": retrieved}



In [24]:
example_q = "When did the Renaissance start?"
out = rag_answer(example_q, k=3)
print("Answer:\n", out["answer"], "\n")

for s in out["snippets"]:
    print("Source:", s["meta"]["source"], "Score:", round(s["score"], 3))
    print(s["text"][:200].replace("\n", " "), "\n")

Answer:
 The Renaissance started in the 14th century.
 

Source: data\text1.txt Score: 0.763
The Renaissance was a period of cultural, artistic, and scientific rebirth that began in Italy during the 14th century. Thinkers like Leonardo da Vinci and Michelangelo transformed art and science. Th 

Source: data\text2.txt Score: 0.466
The Industrial Revolution started in Britain in the 18th century. It was characterized by new manufacturing processes and the invention of machinery such as the steam engine. This era dramatically cha 

Source: data\text2.txt Score: -3.4028234663852886e+38
The Industrial Revolution started in Britain in the 18th century. It was characterized by new manufacturing processes and the invention of machinery such as the steam engine. This era dramatically cha 

